Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<div class="alert alert-block alert-warning">
<b>Комментарий ревьюера</b>

Введение хорошее, однако было бы здорово добавить название проекта и дать содержание/шаги проекта.

</div>


## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.duplicated().sum()

153

In [5]:
data = data.drop_duplicates().reset_index(drop= True)

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера</b>

Хороший первый взгляд на данные. Также в начале полезно провести небольшой анализ данных (несмотря на то, что проект небольшой и в целом это не требуется). Нет ли каких-либо аномальных значений, например, в возрасте или зарплате, как распределен целевой признак.

</div>

## 2. Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
a. Изменится. Приведите примеры матриц.
b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Решение:

$$
((XP)^T XP)^{-1} (XP)^T y = (P^T X^T XP)^{-1} (P^T X^T)y = ((P^T X^T )XP)^{-1} (P^T X^T)y = (XP)^{-1} (P^T X^T)^{-1} (P^T X^T )y = (XP)^{-1} Ey =  (XP)^{-1} y
$$


<font color='orange'>
Хм, по доказательству, приведенному тобой, записываю ниже результат:


$$
((XP)^T XP)^{-1} (XP)^T y = (P^T X^T XP)^{-1} (P^T X^T)y = (P^T (X^T X)P)^{-1} (P^T X^T)y = P^{-1} (X^T X)^{-1} (P^T )^{-1} (P^T X^T )y = P^{-1} (X^T X)^{-1} E (X^T y) = P^{-1} (X^T X)^{-1} (X^T y)
$$

</font>

Соответственно:

$$
w = (XP)^{-1} y
$$

<div class="alert alert-block alert-warning">
<b>Комментарий ревьюера (v2)</b>

Преобразования теперь верны, и тогда получается, что это выражение неверное $w = (XP)^{-1} y$. По итогу новый вектор весов $w_P$ у нас имеет такой вид:

$$
w_P = P^{-1} (X^T X)^{-1} (X^T y) =  P^{-1} (X^T X)^{-1} X^T y
$$

Правая часть выражения это есть изначальный вектор весов $w = (X^T X)^{-1} X^T y$. Итого имеем:

$$
w_P =  P^{-1} (X^T X)^{-1} X^T y =  P^{-1} w
$$

Матрица признаков после умножения на матрицу $P$ имеет такой вид $X_P = XP$. Подставив данные выражения в формулу предсказания имеем:
    
$$
a_P =  XP P^{-1} w = X E w = Xw = a \\
a_P = a
$$

</div>


**Ответ:** b. Не изменится

**Обоснование:** С помощью преобразований получаем результат формулы регрессии y = wXP , то есть в формуле отсутствует сдвиг и присутствует матрица, на которую домножали. 

<div class="alert alert-block alert-danger">
<b>Комментарий ревьюера</b>

<s>Ответ верный, однако в обосновании есть некоторые неточности. Привожу ниже формулы.

    
<div  align='center'>
<p style='border: 2px solid black; padding: 15px; width: 30%' >
$$
(A \cdot B)^{-1} = B^{-1} \cdot A^{-1} \\
(A^T)^{-1} = (A^{-1})^T
$$
</p>
</div>
    
    
<div style='margin: 5px' align='center'>
<p style='border: 2px solid black; padding: 15px; width: 30%' >
$$
A \cdot (B \cdot C) = (A \cdot B) \cdot C \\
A \cdot B \neq B \cdot A \\
(A \cdot B)^T = B^T \cdot A^T \\
$$
</p>
</div>
    
Но так как матрицы $X$ и $X^T$, а также $(XP)^T$ и $XP$, прямогульные, то обратных матриц у них нет, то есть открывать скобки так нельзя:
    
$$
(X^T X)^{-1} \neq (X^T)^{-1} X^{-1}
$$  
    
$$
((XP)^T(XP))^{-1} \neq ((XP)^T)^{-1}(XP)^{-1}
$$

Здесь для второго выражения более правильный подход такой. То есть  после того как раскрывается внутреннее транспонирование сначала группируем в три квадратные матрицы, а потом раскрываем скобки:

$$
((XP)^T(XP))^{-1} = (P^T  X^T  X  P)^{-1} = (P^T  (X^T  X)  P)^{-1} = P^{-1}  (X^T  X)^{-1}  (P^T)^{-1}
$$

    
</div>


## 3. Алгоритм преобразования

Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм**

Алгоритм шифрования - это умножение матрицы признаков на матрицу Р, которая квадратна и обратима.

<div class="alert alert-block alert-warning">
<b>Комментарий ревьюера</b>

Здесь преполагается записать алгоритм в таком виде:

1. Создадим...
2. Проверим...
3. Умножим... и т.д.

</div>

**Обоснование**

Обоснование - доказательство в шаге 2.

## 4. Проверка алгоритма

Здесь создала рандомно матрицу, на которую умножала наши признаки для проверки модели. Обучила модель на изначальных и зашифрованных признаках, r2 получился одинаковый(с минимальными погрежностями)


In [7]:
matrix1 = np.random.normal(size=(4,4)) 

In [8]:
np.linalg.inv(matrix1)

array([[ 3.83722618, -1.13068861, -0.14782676, -2.87029953],
       [-4.33130539,  1.88741455,  0.65174788,  3.34713601],
       [ 1.30489549, -0.55553322, -0.9063947 , -0.80245457],
       [ 1.4697278 , -0.49397623,  0.46577949, -0.40090971]])

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера</b>

Отлично, проверка на обратимость есть! Было здорово реализовать ее генерацию и проверку на обратимость с помощью функции или цикла (то есть если сгенерировалась необратимая матрица, то генерируем другую автоматически).

</div>


<div class="alert alert-block alert-danger">
<b>Комментарий ревьюера</b>

<s>Однако в данном случае правильнее создать рандомную матрицу и проверить ее на обратимость. Данные же умножать на изначальную матрицу. А чтобы расшифровать данные - надо зашифрованные данные умножить на обратную матрицу.

</div>


<div class="alert alert-block alert-success">
<b>Комментарий ревьюера (v2)</b>

👍

</div>


<font color='orange'> Все, данные умножаются на исходную матрицу

In [9]:
features = data.drop(['Страховые выплаты'], axis = 1)
target = data['Страховые выплаты']

In [10]:
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4842,0,28.0,35700.0,2
4843,0,34.0,52400.0,1
4844,0,20.0,33900.0,2
4845,1,22.0,32700.0,3


In [11]:
features1 = features@matrix1

In [12]:
features1.columns=['Пол', 'Возраст', 'Зарплата', 'Члены семьи']

In [13]:
def x(features,target):
    features_train,features_valid,target_train,target_valid = train_test_split(features, target, test_size = 0.25,random_state=12345)
    model = LinearRegression()
    model.fit(features_train,target_train)
    prediction=model.predict(features_valid)
    result = r2_score(target_valid,prediction)
    print('r2', result)

In [14]:
x(features,target)

r2 0.42307727492147296


In [15]:
x(features1,target)

r2 0.42307727492147396


<div class="alert alert-block alert-success">
<b>Комментарий ревьюера</b>

Результат подтвержден экспериментально, осталось математически 😉

</div>


<div class="alert alert-block alert-danger">
<b>Комментарий ревьюера</b>

<s>Пожалуйста, добавь краткий вывод по итогам проекта. 

</div>


# <font color='violet'>Вывод

<font color='violet'> В данном проекте стояла задача защитить лданные с помощью шифрования. Задача была успешно реализована посредством выполения умножения данных на рандомную матрицу с требованиями обратимости и квадратности. К данному способу шифрования наводил сам проект в самом начале, посредством проведенных теоретических доказательств было утсановлено, что данные, при умножении на такую матрицу, не поменяются, что мы и подтвердили в дальнейшем на практике при проверке метрики r2 нашей модели. 

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера (v2)</b>

Отлично.

</div>


## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования